# Notebook V - Terrain Constraints

<hr>
This module calculates the yield reduction due to terrain slope, soil erosion, and Fournier Index (FI). Note that starting from v2.2, all terrain reduction factors are to be provided as excel sheets.

Prepared by Geoinformatics Center, AIT
<hr>

### Google drive connection
In this step, we will connect to Google Drive service and mount the drive where we will start our PyAEZ project

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Then, installing any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [ ]:
# 'Installing neccessary packages'
# !pip install gdal
# # !pip install pyaez==2.1.0

## Importing Libraries

In [ ]:
'''import supporting libraries'''
# import pyaez
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
try:
    from osgeo import gdal
except:
    import gdal
import sys

Setting the working directory -- where our PyAEZ project is located.

In [ ]:
'Set the working directory'
work_dir = r'D:\test_working_folder'  # Please change this to your working directory
os.chdir(work_dir)
sys.path.append('D:\PyAEZ_iiasa')
os.getcwd()


Check and create data output folder

In [ ]:
import os
folder_path = './data_output/NB5/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")


## Applying Terrain Constraints

In [ ]:
'''importing library'''

from pyaez import TerrainConstraints
terrain_constraints = TerrainConstraints.TerrainConstraints()

from pyaez import UtilitiesCalc
obj_utilities = UtilitiesCalc.UtilitiesCalc()

In [ ]:
'''reading climate and slope data'''
basepath = r'D:\PyAEZ_iiasa\data_input\LAO_Admin.tif'
precipitation = np.load(r'D:\PyAEZ_iiasa\data_input\climate/precipitation.npy') # mm / day

slope_map = gdal.Open(r'D:\PyAEZ_iiasa\data_input\LAO_Slope.tif').ReadAsArray() # Percentage Slope

New Feature: Importing Terrain Reduction Factor Setting as Excel Sheets

In [ ]:
terrain_constraints.importTerrainReductionSheet(irr_file_path=r'D:\PyAEZ_iiasa\data_input\maiz_terrain_constraints_irr.xlsx',
                                rain_file_path=r'D:\PyAEZ_iiasa\data_input\maiz_terrain_constraints_rain.xlsx')


In [ ]:
'''passing climate and slope data'''

terrain_constraints.setClimateTerrainData(precipitation, slope_map)

In [ ]:
'''calculation of Fournier index'''
terrain_constraints.calculateFI()

# extraction of Fournier index (FI) if required
fi = terrain_constraints.getFI()

In [ ]:
# Visualizing the outputs
plt.imshow(fi)
plt.colorbar()
plt.title('Fournier Index')

In [ ]:
# saving FI data
obj_utilities.saveRaster(basepath, r'./data_output/NB5/fournier_index.tif',fi)

In [ ]:
'''reading yield data'''
yield_map_rain = gdal.Open('./data_output/NB4/soil_clim_yld_maiz_rain.tif').ReadAsArray()

Applying Terrain Constraints for Rainfed Condition

In [ ]:
'''applying terrain constraints (Rainfed)'''

yield_map_rain_m5 = terrain_constraints.applyTerrainConstraints(yield_map_rain, 'R') # I: Irrigated, R: Rain-fed

## get classified output
# yield_map_rain_class_m5 = obj_utilities.classifyFinalYield(yield_map_rain_m5)

# getting the terrain reduction factor (fc5)
fc5_rain = terrain_constraints.getTerrainReductionFactor()

In [ ]:
'''visualize result'''

plt.figure(1, figsize=(18,9))
plt.subplot(1,2,1)
plt.imshow(yield_map_rain, vmax = np.max([yield_map_rain_m5, yield_map_rain]))
plt.colorbar(shrink=0.8)
plt.title('Original Rainfed Yield Maize')

plt.subplot(1,2,1)
plt.imshow(yield_map_rain_m5, vmax = np.max([yield_map_rain_m5, yield_map_rain]))
plt.colorbar(shrink=0.8)
plt.title('Terrain Constrainted Rainfed Yield Maize')

plt.subplot(1,2,3)
plt.imshow(fc5_rain, vmax = 1, vmin = 0)
plt.colorbar(shrink=0.8)
plt.title('Terrain Constrainted Rainfed Yield Maize')

In [ ]:
# saving rainfed outputs
obj_utilities.saveRaster(basepath, r'./data_output/NB5/terr_soil_clim_yld_maiz_rain.tif', yield_map_rain_m5)
obj_utilities.saveRaster(basepath, r'./data_output/NB5/fc5_maiz_rain.tif', fc5_rain)

Applying Terrain Constraints for Irrigated Condition

In [ ]:
'''reading Irrigated yield data'''

yield_map_irr = gdal.Open('./data_output/NB4/soil_clim_yld_maiz_irr.tif').ReadAsArray()

In [ ]:
'''applying terrain constraints (Irrigated)'''
yield_map_irr_m5 = terrain_constraints.applyTerrainConstraints(yield_map_irr, 'I') # I: Irrigated, R: Rain-fed

## get classified output

# yield_map_irr_class_m5 = obj_utilities.classifyFinalYield(yield_map_irr_m5)

# getting the terrain reduction factor (fc5)
fc5_irr = terrain_constraints.getTerrainReductionFactor()

In [ ]:
'''visualize result'''

plt.figure(1, figsize=(18,9))
plt.subplot(1,2,1)
plt.imshow(yield_map_irr, vmax = np.max([yield_map_irr_m5, yield_map_irr]))
plt.colorbar(shrink=0.8)
plt.title('Original Irrigated Yield Maize')

plt.subplot(1,2,1)
plt.imshow(yield_map_irr_m5, vmax = np.max([yield_map_irr_m5, yield_map_irr]))
plt.colorbar(shrink=0.8)
plt.title('Terrain Constrainted Irrigated Yield Maize')

plt.subplot(1,2,3)
plt.imshow(fc5_irr, vmax = 1, vmin = 0)
plt.colorbar(shrink=0.8)
plt.title('Terrain Constrainted Irrigated Yield Maize')

In [ ]:
# saving irrigated outputs
obj_utilities.saveRaster(basepath, r'./data_output/NB5/terr_soil_clim_yld_maiz_irr.tif', yield_map_irr_m5)
obj_utilities.saveRaster(basepath, r'./data_output/NB5/fc5_maiz_irr.tif', fc5_irr)

<hr>

### END OF MODULE 5: TERRAIN CONSTRAINTS

<hr>